## Model Training Example
This notebook demonstrates saving and loading a tensorflow model into and out of a class.

It implements a class to store the basic skeleton of a tensorflow model, but leaves out any training or prediction methods.

In [1]:
import tensorflow as tf
import numpy as np

### Model Class

We can use classes to encapsulate tensorflow models. The below skeleton shows one way of using classes with tensorflow.

The Model class has methods, variables, and properties that capture both the graph and the tensorflow session

#### Tensorflow Graph

A tensorflow graph is a computational graph of different tensorflow operations. It defines the computation and how different operations and tensors relate, but it doesn't actually do the computation or store the values of the variables. All of that magic happens within the tensorflow session.

#### Tensorflow Session

A tensorflow session is the context where values for tensorflow variables are instantiated and computations are run. So if you are saving a model's weights, you are actually saving the weights of the tensorflow session. If you are loading a model's weights, you need to load them into a session. When variables are initialized, that has to happen within a session. In a way, the graph is stateless. State is stored in sessions. The session also takes care of running computations, so if you are running training, those need to be run in the session.

A session is instantiated with a graph, typically the current default graph. A session is only able to run computations on the graph that is tied to the session.

In [2]:
class DeepLearningModel():
    def __init__():
        return
    
    def gen_uniform_random_weights(self, k_out, k_in, scale, dtype=np.float32):
        """
        Returns weights of shape (k_in, k_out) initialized between [-scale, scale]
        """
        return ((np.random.rand(k_in, k_out) * 2 - 1) * scale).astype(dtype)

    def gen_random_weights_tanh(self, k_out, k_in, dtype=np.float32):
        scale = (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_sigmoid(self, k_out, k_in, dtype=np.float32):
        scale = 4. * (6. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_random_weights_reLu(self, k_out, k_in, dtype=np.float32):
        scale = (2. / (k_in + k_out)) ** .5
        return self.gen_uniform_random_weights(k_out, k_in, scale, dtype=dtype)

    def gen_biases(self, k, dtype=np.float32):
        """
        Initialize biases as zero.
        """
        return np.zeros((k, ), dtype=dtype)
    
    def clip_gradient(self, grad, magnitude=1.0):
        """returns a clipped gradient, where it is between [-magnitude and magnitude]"""
        magnitude = abs(magnitude)
        return tf.maximum(tf.minimum(grad, magnitude), - magnitude)

In [31]:
class RecurrentNetworkModel(DeepLearningModel):
    """
    Tutorial Model
    """
    
    def __init__(self, num_layers, k_rnncell, k_input_embedding, number_data_embeddings, time_steps):
        """
        args:
            num_layers: number of hidden layers
            k_hidden: number of units in the hidden layers
            k_input_embedding: dimensionality of the input
            k_softmax: dimensionality of the output layer
        """
        self._graph = None
        self._session = None
        self.num_layers = num_layers
        self.k_rnncell = k_rnncell
        self.k_input_embedding = k_input_embedding
        self.time_steps = time_steps
        self.number_data_embeddings = number_data_embeddings
        
        self._merged_training_summary = None
        self._merged_validation_summary = None
    
    
    def load_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
        
        self._session = tf.Session(graph=self.graph)
        model_saver.restore(self._session, model_filename)
        return
    
    
    def save_model(self, model_filename):
        with self.graph.as_default():
            model_saver = tf.train.Saver()
            
        model_saver.save(self.session, model_filename)
        
    def create_graph(self):
        self.cells = {}
                        
        self._graph = tf.Graph()
        with self._graph.as_default():
            with tf.name_scope("inputs"):
                self.X = tf.placeholder(tf.int32, shape=(None, self.time_steps))
            
            with tf.name_scope("embeddings"):
                self.data_embeddings = tf.Variable(np.random.rand(self.number_data_embeddings, self.k_input_embedding).astype(np.float32), dtype=tf.float32, name="data_embeddings")
                self.data_embedding_lookup = tf.nn.embedding_lookup(self.data_embeddings, self.X)
            
            with tf.name_scope("recurrent_layers"):
                self.cells = []
                for layer in range(1, num_layers + 1):
                    cell = tf.contrib.rnn.BasicRNNCell(self.k_rnncell)
                    self.cells.append(cell)
                self.rnn_cells = tf.contrib.rnn.MultiRNNCell(self.cells)
                
                self.rnn_output, self.rnn_state = tf.nn.dynamic_rnn(self.rnn_cells, self.data_embedding_lookup, dtype=tf.float32)
                
            self.init_op = tf.global_variables_initializer()
        return
            

    def write_graph(self):
        self.tensorboard_writer.add_graph(self.graph)
        return
    
    def init_model(self, adam_beta1=0.9, adam_beta2=0.999):
        self.session.run(self.init_op)
    
    @property
    def graph(self):
        if self._graph is None:
            self.create_graph()
        return self._graph
    
    @property
    def session(self):
        if self._session is None:
            self._session = tf.Session(graph=self.graph)
        return self._session

In [48]:
num_layers = 2
k_rnncell = 7
k_input_embedding = 12
number_data_embeddings = 100
time_steps = 10

In [49]:
# create a model instance with 2 hidden layers and 10 hidden units.

model_a = RecurrentNetworkModel(num_layers,  k_rnncell, k_input_embedding, number_data_embeddings, time_steps)

In [50]:
model_a.graph

In [51]:
model_a.data_embedding_lookup

<tf.Tensor 'embeddings/embedding_lookup:0' shape=(?, 10, 12) dtype=float32>

In [52]:
model_a.data_embedding_lookup

<tf.Tensor 'embeddings/embedding_lookup:0' shape=(?, 10, 12) dtype=float32>

In [53]:
model_a.init_model()

In [54]:
model_a.data_embedding_lookup.get_shape()

TensorShape([Dimension(None), Dimension(10), Dimension(12)])

In [55]:
model_a.session.run(model_a.data_embeddings).shape

(100, 12)

In [56]:
data = np.random.randint(0, number_data_embeddings, size=(4, time_steps))

In [57]:
model_a.session.run(model_a.data_embedding_lookup,
                    feed_dict={model_a.X: data}).shape

(4, 10, 12)

In [58]:
model_a.session.run(model_a.rnn_output,
                    feed_dict={model_a.X: data}).shape

(4, 10, 7)

In [59]:
model_a.session.run(model_a.rnn_output,
                    feed_dict={model_a.X: data}).shape

(4, 10, 7)

---
When using a 2 layer network, the state is the final state after 

In [65]:
model_a.session.run(model_a.rnn_state,
                    feed_dict={model_a.X: data})

(array([[-0.48340967, -0.61855316, -0.58779192,  0.63455355,  0.08212776,
          0.82205856, -0.21148086],
        [-0.32181871, -0.83180571, -0.51363915,  0.88018322,  0.12627649,
          0.71377528,  0.05713412],
        [-0.33939523, -0.64917535, -0.52595317,  0.74828136,  0.04690533,
          0.59514552, -0.55521268],
        [-0.63948739, -0.80748004, -0.24147934,  0.56942451,  0.07888543,
          0.86686981, -0.06308942]], dtype=float32),
 array([[ 0.1622695 ,  0.06218038,  0.57559121, -0.46408698, -0.35307229,
         -0.02346844, -0.54259211],
        [ 0.10517123,  0.28516692,  0.78869605, -0.21677148, -0.24923387,
         -0.16148129, -0.65151817],
        [ 0.49877018, -0.28370455,  0.5651896 , -0.4388313 , -0.31489947,
         -0.19673066, -0.48347014],
        [ 0.23157886, -0.1996063 ,  0.47212562, -0.33287796, -0.15090299,
         -0.17515591, -0.36926678]], dtype=float32))

In [63]:
model_a.session.run(model_a.rnn_output,
                    feed_dict={model_a.X: data}).shape

(4, 10, 7)

---
The output above shows how to get the rnn_output, it's fairly straightforward to append an output model on the end of the final rnn_states or the rnn_outputs.